# Dimensionality reduction

- PCA & UMAP 
- Clustering using spectral clustering

In [1]:
# load in required modules
import gzip, pandas as pd, numpy as np, requests, os, gc
from urllib.request import urlretrieve
import os
import zipfile

### Load in the data from online location

In [2]:
# download and load GTEx data files
files = {
    # GTEx gene expression data
    'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz': 'https://storage.googleapis.com/adult-gtex/bulk-gex/v8/rna-seq/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz',
    
    # GTEx metadata files
    'GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt': 'https://storage.googleapis.com/adult-gtex/annotations/v8/metadata-files/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt',
    'GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt': 'https://storage.googleapis.com/adult-gtex/annotations/v8/metadata-files/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt'
}

# download files 
for filepath, url in files.items():
    if not os.path.exists(filepath):
        try:
            urlretrieve(url, filepath)
        except Exception as e:
            print(f"Error: {e}")
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
            else:
                print(f"Failed to download. {response.status_code}")
    else:
        print(f"{filepath} already exists")

### PCA

### UMAP

### PCA -> Spectral clustering

### UMAP -> Spectral clustering